In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
training_data = pd.read_csv('train.csv') #188533 rows, 13 columns - last column is price
test_data = pd.read_csv('test.csv') #125690 rows
test_data['price'] = 0
all_data = pd.concat([training_data, test_data], ignore_index = True)
#clean_title, accident, fuel_type have "NaN" types
all_data = all_data.drop(columns = ["id", "model", "ext_col", "int_col", "clean_title", "engine"])
columns_to_process = ['brand', 'fuel_type', 'transmission']
all_data.fillna({'fuel_type' : 'unknown'}, inplace = True)
for column in columns_to_process:
    mean_values = all_data.groupby(column)['price'].mean()
    all_data[column] = all_data[column].map(mean_values)
X_data = all_data.iloc[:, :-1]
y_data = all_data.iloc[:, -1]
X_data = pd.get_dummies(X_data, columns=['accident'], prefix='accident')
X_data.rename(columns={'accident_At least 1 accident or damage reported': 'accident1'}, inplace=True)
X_data.rename(columns={'accident_None reported': 'accident0'}, inplace=True)
X_data = X_data.astype(int)
scaler = StandardScaler()
X_data = pd.DataFrame(scaler.fit_transform(X_data), columns=X_data.columns)

In [3]:
X_train = X_data.head(188533)
y_train = y_data.head(188533)
X_test = X_data.tail(125690)

In [154]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
X_train = X_data.head(188533)
y_train = y_data.head(188533)
model.fit(X_train, y_train)
X_test = X_data.tail(125690)
y_pred = model.predict(X_test)
id = np.array([idx for idx in range(188533, 314223)])
prediction = pd.DataFrame({'id' : id, 'price' : y_pred})

In [156]:
id = np.array([idx for idx in range(188533, 314223)])
prediction = pd.DataFrame({'id' : id, 'price' : y_pred})

In [ ]:
prediction.to_csv('09052024carsprediction2.csv', index = False)